In [106]:
import csv #loading csv package
import pandas as pd #loading pandas package
import re #loading regex package
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
from scipy.stats import norm
from scipy.stats import hypergeom 
from bokeh.models import Span
from bokeh.resources import CDN
from bokeh.embed import file_html, components
from bokeh.plotting import figure, ColumnDataSource, output_notebook, show, output_file
from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool
from bokeh.palettes import brewer

# Contents

1. Reading in the user input file
2. Cleaning up the user input file
3. Obtaining kinase relationship information from the database
4. Carrying out delta count calculation method of KSEA activity
    a)KSEA mean method
    b)KSEA alternative mean method
    c)Delta Count method
5. Visualising the data

In [285]:
def ReadDataInput(userDataInput):

    #read in txt file
    df_input_original = pd.read_csv(userDataInput,  sep='\t')

    #There are 86 columns in the dataframe, but only 7 columns have values, the rest are empty
    #Need to remove the empty columns
    input_original_subset = df_input_original.iloc[:, 0:7]

    #Make columns 2-7 type float instead of string
    input_original_subset.iloc[:, 1:7] = input_original_subset.iloc[:, 1:7].astype(float)

    #Need to separate the phosphosite from the substrate in the first column into 2 separate columns
    input_original_subset[['Substrate','Phosphosite']] = input_original_subset.Substrate.str.split('\(|\)', expand=True).iloc[:,[0,1]]

    #Remove any rows where there are NaN in any of the columns
    input_original_subset=input_original_subset.dropna()

    return input_original_subset

input_original_subset=ReadDataInput('az20.txt')

,Substrate,control_mean,AZ20_mean,AZ20_fold_change,AZ20_p-value,AZ20_ctrlCV,AZ20_treatCV,Phosphosite
0,1A24_HUMAN,1.527934e+07,2.643439e+07,1.730074,0.554298,1.280092,0.902944,S356
1,1A24_HUMAN,1.527934e+07,2.643439e+07,1.730074,0.554298,1.280092,0.902944,S359
2,1B13_HUMAN,2.706473e+07,2.779116e+07,1.026841,0.962084,0.724637,0.580207,None
3,1B39_HUMAN,1.158130e+09,1.173595e+09,1.013353,0.958301,0.128036,0.399220,M1
4,1B39_HUMAN,1.158130e+09,1.173595e+09,1.013353,0.958301,0.128036,0.399220,M4


In [286]:
#Read in human kinase substrate and phosphosite csv
#Add unique identifier for each unique p-site 
def SubstrateKinase(SubstrateKinaseCSV):
    df_substrate = pd.read_csv(SubstrateKinaseCSV)
    df_substrate['p-site'] = df_substrate['SUB_ACC_ID'] +'_' + df_substrate['SUB_MOD_RSD']
    return df_substrate

df_substrate=SubstrateKinase('new_clean_human_kinase_substrates.csv')

In [287]:
#Obtain adjacency matrix of phosphosites to kinase relationships
#1 means phsophorylation by a kinase
#0 means no relationship
#then sum number of relationships per kinase
def Adj_Matrix(df_substrate):
    df_substrate["value"] = 1
    rel_adj_matrix = pd.pivot_table(df_substrate, values='value' , index='p-site' , columns='GENE',fill_value=0) 
    rel_adj_matrix.sum(axis=0).sort_values(ascending=False).head()

Adj_Matrix(df_substrate)

In [288]:
#Finding the kinase-substrate interactions
def SubstrateKinaseInteractions(input_original_subset):
    input_original_subset["Kinase"] = ""
    length=len(input_original_subset)

    for i,j, k in zip(input_original_subset['Substrate'],input_original_subset['Phosphosite'], range(length)):
        for l, m, n, o, p in zip(df_substrate['SUB_ENTRY_NAME'], df_substrate['SUBSTRATE'],  df_substrate['SUB_GENE'],df_substrate['SUB_MOD_RSD'], df_substrate['KINASE']):
            if (i == l or i == m or i == n) and j == o:
                input_original_subset.iloc[k ,8] += p + ","
    Kin_Sub_Interactions=input_original_subset
    return Kin_Sub_Interactions

Kin_Sub_Interactions=SubstrateKinaseInteractions(input_original_subset)

In [289]:
#There are empty values in the new kinase column in the user input file
#These empty rows are where there was not a match - either because the phosphosite in the user input 
#didn't match our database, or where there was not phosphosite stated in the user input file
#Remove rows where there are empty values in the kinase column

def TidyKinaseSubstrate(Kin_Sub_Interactions):
    #First, need to replace the empty values with NaN
    nan_replace = float("NaN")
    Kin_Sub_Interactions.replace("", nan_replace, inplace=True)

    #then can use drop.na() to drop rows where 'NaN' appears
    user_input_kinase=Kin_Sub_Interactions.dropna(subset = ["Kinase"])

    return user_input_kinase

tidy_input_kinase=TidyKinaseSubstrate(Kin_Sub_Interactions)

In [293]:
def splitDataFrameList(df,target_column,separator):
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    nan_replace = float("NaN")
    new_df.replace("", nan_replace, inplace=True)
    kinaseList=new_df.dropna(subset = ["Kinase"])
    
    ##Drop ambiguous phosphosites - phosphosites that are phosphorylated by multiple kinases
    kinaseList.drop_duplicates(subset=['Phosphosite', 'Substrate'], keep=False, inplace=True)
    kinaseList=kinaseList.reset_index(drop=True)
    return kinaseList

kinaseList = splitDataFrameList(tidy_input_kinase, 'Kinase', ',')
kinaseList.head()

,AZ20_ctrlCV,AZ20_fold_change,AZ20_mean,AZ20_p-value,AZ20_treatCV,Kinase,Phosphosite,Substrate,control_mean
0,0.339347,0.831641,2.678659e+08,0.572706,0.423268,CDK1,T389,AAK1,3.220931e+08
1,0.184789,1.578449,5.887469e+09,0.024749,0.178244,CK2A1,S109,ABCF1,3.729907e+09
2,0.209021,0.752334,3.859859e+09,0.171288,0.239829,CK2A1,S140,ABCF1,5.130510e+09
3,0.043769,1.349337,4.794626e+08,0.121853,0.246919,Abl,Y213,ABI1,3.553320e+08
4,1.001082,1.791091,5.593549e+07,0.446615,0.755225,CDK1,S12,ADD1,3.122984e+07


In [304]:
def Accession_Add(df_substrate,kinaseList):
    kinaseList["SUB_ACC_ID"]=""
    length=len(kinaseList)

    for i,j,k in zip(kinaseList['Phosphosite'],kinaseList['Kinase'], range(length)): 
        for l, m, n in zip(df_substrate['SUB_MOD_RSD'], df_substrate['KINASE'], df_substrate['SUB_ACC_ID']):    
            if i == l and m == j:
                kinaseList.at[k,'SUB_ACC_ID'] = n

Accession_Kinase_df=Accession_Add(df_substrate,kinaseList)
#kinaseList.head()

In [305]:
def Unique_identifier_Kinase(df):
    kinaseList['p-site'] = kinaseList['SUB_ACC_ID'] +'_' + kinaseList['Phosphosite']
    return kinaseList

Unique_identifier_kinase=Unique_identifier_Kinase(kinaseList)

In [306]:
##Find how many phosphosites are found in both the substrate set and the kinase database 

def phosphosite_in_common(rel_adj_matrix, kinaseList):
    common_phosphosite=[]
    
    for i in rel_adj_matrix:
        substrate_set = rel_adj_matrix[i].replace(0, np.nan).dropna().index
        detected_p_sites = kinaseList.iloc[:,10]
        intersect = list(set(substrate_set).intersection(detected_p_sites))
        common_phosphosite.append(intersect)
    
    return common_phosphosite

common_phosphosite=phosphosite_in_common(rel_adj_matrix, kinaseList)

def len_intersect(common_phosphosite):
    len_intersect=[]
    for i in common_phosphosite:
        len_intersect.append(len(i))
    return len_intersect

len_intersect= len_intersect(common_phosphosite)

gene_list=[] 
for i in rel_adj_matrix:
    gene_list.append(i)
    
Phosphosite_number = {'Substrate':gene_list,'P-site Number':len_intersect}
Phosphosite_no=pd.DataFrame(Phosphosite_number)

In [302]:
##Carry out -log10 transform on P values
def NegLog10(df):
    
    #Take -log10 of the corrected p-value.
    uncorrected_p_values=df.iloc[ :,3].astype(np.float64)
    log10_corrected_pvalue = (-np.log10(uncorrected_p_values))

    #Append -log10(P-values) to a new column in data frame.
    df["-Log10 Corrected P-Value"]=log10_corrected_pvalue
    NegLog10kinase=df
    return NegLog10kinase

NegLog10KinaseDF=NegLog10(Unique_identifier_kinase)

In [313]:
#Calculate log2FC and add as new column
def log2FC(df):
    log2FC=np.log2(df.iloc[:, 1])
    df["Log2 Fold Change"]=log2FC
    return df
log2FCKinase= log2FC(NegLog10KinaseDF)

# KSEA "Mean" Method

MS = Mean fold changes in substrate set 

mP = mean FC of data set

m=size of substrate set

delta= std of fold change of complete data set 

In [314]:
def KSEA_Mean(df):#mS calculation
    mS = df.groupby('Kinase')['Log2 Fold Change'].mean()
    mP = df['Log2 Fold Change'].mean()
    delta=df['Log2 Fold Change'].std()

    m=[]
    Kinase_phosphosite=df.groupby('Kinase')['Phosphosite']
    for key, item in Kinase_phosphosite:
        m.append(len(item))

    Z_Scores=[]    
    for i, j in zip(mS, m):
        Z_Scores.append((i-mP)*math.sqrt(j)*1/delta)

    p_means=[]
    for i in Z_Scores:
        p_means.append(norm.sf(abs(i)))

    calculations_dict={'mS': mS, 'mP':mP, 'm':m, 'Delta':delta, 'Z_Scores':Z_Scores,"P_value":p_means}

    calculations_df=pd.DataFrame(calculations_dict)
    calculations_df=calculations_df.reset_index(level=['Kinase'])
    return calculations_df

calculations_df=KSEA_Mean(log2FCKinase)

# Alternative Mean method

In the alternative mean method, only the phosphosites in the substrate set that change signiﬁcantly between conditions are considered when calculating the mean of the fold changes in the substrate set. 

Therefore, a cut off is needed, where only phosphosites in the substrate set with a signiﬁcant increase or decrease are used.

In [317]:
print new_kinaseList

     AZ20_ctrlCV  AZ20_fold_change     AZ20_mean  AZ20_p-value  AZ20_treatCV  \
1       0.184789          1.578449  5.887469e+09      0.024749      0.178244   
5       0.312763          0.212892  3.721304e+08      0.007357      0.664823   
12      0.322419          1.666240  3.181032e+08      0.023345      0.122824   
15      0.284972          0.356389  2.176100e+08      0.008361      0.131286   
28      0.273054          2.472960  2.004610e+09      0.005000      0.211943   
30      0.086407          1.421471  7.041092e+09      0.000221      0.023482   
43      0.146188          0.358882  4.864442e+08      0.000440      0.179713   
46      0.248967          1.943308  7.233623e+07      0.034177      0.280033   
56      0.034670          1.581005  5.059355e+09      0.000998      0.104506   
57      0.194614          1.801883  5.511817e+09      0.043778      0.282943   
65      0.079779          1.524510  4.559793e+09      0.004012      0.120968   
66      0.137253          1.323145  1.36

In [334]:
def KSEA_alt_mean(kinaseList):
    #First, need to set cut off for -log10 corrected p values
    #the cut off will be -log10 transformed 0.05 significance 
    cutOff = -np.log10(0.05)
    #Filter kinase-phosphosites by the significance cut off
    new_kinaseList=kinaseList.loc[kinaseList.iloc[:,11] > cutOff]

    mP = kinaseList['Log2 Fold Change'].mean()
    delta=kinaseList['Log2 Fold Change'].std()
    
    #Calculate m for reduced dataset
    alt_m=[]
    Kinase_phosphosite_alt=new_kinaseList.groupby('Kinase')['Phosphosite']
    for key, item in Kinase_phosphosite_alt:
        alt_m.append(len(item))

    #Calculation alternative mS
    alt_mS=new_kinaseList.groupby('Kinase')['Log2 Fold Change'].mean()

    #Calculate Z score:
    alt_z_scores=[]    
    for i, j in zip(alt_mS, alt_m):
        alt_z_scores.append((i-mP)*math.sqrt(j)*1/delta)

    #calculate alternative p value mean from z score
    alt_p_means=[]
    for i in alt_z_scores:
        alt_p_means.append(norm.sf(abs(i)))

    #Make dataframe for calculations
    alt_calculations_dict={'mS': alt_mS, 'mP':mP, 'm':alt_m, 'Delta':delta, 'Z_Scores':alt_z_scores,"P_value":alt_p_means}

    alt_calculations_df=pd.DataFrame(alt_calculations_dict)
    alt_calculations_df=alt_calculations_df.reset_index(level=['Kinase'])
    return alt_calculations_df

alt_calculations=KSEA_alt_mean(log2FCKinase)

,AZ20_ctrlCV,AZ20_fold_change,AZ20_mean,AZ20_p-value,AZ20_treatCV,Kinase,Phosphosite,Substrate,control_mean,SUB_ACC_ID,p-site,-Log10 Corrected P-Value,Log2 Fold Change,color
0,0.339347,0.831641,2.678659e+08,0.572706,0.423268,CDK1,T389,AAK1,3.220931e+08,Q2M2I8,Q2M2I8_T389,0.242068,-0.265967,grey
1,0.184789,1.578449,5.887469e+09,0.024749,0.178244,CK2A1,S109,ABCF1,3.729907e+09,Q8NE71,Q8NE71_S109,1.606443,0.658508,Red
2,0.209021,0.752334,3.859859e+09,0.171288,0.239829,CK2A1,S140,ABCF1,5.130510e+09,P17936,P17936_S140,0.766272,-0.410554,grey
3,0.043769,1.349337,4.794626e+08,0.121853,0.246919,Abl,Y213,ABI1,3.553320e+08,Q8IZP0,Q8IZP0_Y213,0.914164,0.432251,grey
4,1.001082,1.791091,5.593549e+07,0.446615,0.755225,CDK1,S12,ADD1,3.122984e+07,P35611,P35611_S12,0.350066,0.840839,grey


# The 'Delta Count' method of calculating KSEA

In the ’Delta count’ method, the number of phosphosites that are signiﬁcantly decreased in the condition versus the control are substracted from the number of phosphosites in the substrate set that are signiﬁcantly increased.

The p-value of the score is calculated with a hypergeometric test, since the number of signiﬁcantly regulated phosphosites is a discrete variable.

Need as variables: 
M = the total number of detected phosphosites 
n = the size of the substrate set
N = the total number of phosphosites that are in an arbitrary substrate set and signiﬁcantly regulated. 

In [345]:
cutOff = -np.log10(0.05)
#Filter kinase-phosphosites by the significance cut off
new_kinaseList=log2FCKinase.loc[log2FCKinase.iloc[:,11] > cutOff]

#Make 2 new objects, 1 containining a dataframe where the Fold Change is greater than 1 (upregulated),
#The other containing a dataframe where the FOld Change is less than 1 (downregulated)
upregulated_kinaseList=new_kinaseList.loc[new_kinaseList.iloc[:,12] > 0]
downregulated_kinaseList=new_kinaseList.loc[new_kinaseList.iloc[:,12] < 0]

#Sum the upregulation and downregulation fold changes across the substrate sets for each kinase
upreg_sum_group=upregulated_kinaseList.groupby('Kinase')['Phosphosite'].count()
downreg_sum_group=downregulated_kinaseList.groupby('Kinase')['Phosphosite'].count()

#MAke a dataframe containing the kinases and the the upregulation and downregulation lists for each kinase
#Fill nan with 0 where there were no upregulation/downregulation for a particular kinase
delta_df= pd.concat([upreg_sum_group, downreg_sum_group], axis=1)
delta_df.fillna(0, inplace=True)
delta_df.head()

#Create an empty list that will contain upregulation - downregulation (delta count) for each kinase
sumList=[]
for i,j in zip(delta_df.iloc[:,0], delta_df.iloc[:,1]):
    sumList.append(i-j)
delta_df['Delta_Count']=sumList
delta_df.head()
print len(sumList)
print len(delta_df)
#Hypergeometric test

M = len(log2FCKinase) 
n= []
Kinase_phosphosite=log2FCKinase.groupby('Kinase')['Phosphosite']
for key, item in Kinase_phosphosite:
    n.append(len(item))
hypergeometric_p=[]
for n, N in zip(n ,delta_df['Delta_Count']):
    hypergeometric_test=hypergeom(M, n, N)
    hypergeometric_p.append(hypergeometric_test.pmf(len(new_kinaseList)))
hypergeometric_p

35
35


C:\Users\sheri\Anaconda2\lib\site-packages\ipykernel_launcher.py:16: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




[0.0,
 0.0,
 0.0,
 0.0,
 nan,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 nan,
 nan,
 nan,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
#Carry out a False Discovery Rate correction using Benjamini/Hochberg correction 
#Correct p-value for multiple testing errors
#Pass p-value series to fdrcorrection function of statsmdodels module.

#hypothesis_rej, corrected_pvalue =statsmodels.stats.multitest.fdrcorrection(input_original_subset.iloc[:, 4], alpha=0.05)

#Convert hypothesis_rej and corrected_pvalue to dataframes.
#hypothesis_rej_df = pd.DataFrame(hypothesis_rej)
corrected_pvalue_df = pd.DataFrame(input_original_subset.iloc[:, 4])

# Append hypothesis_rej_df and corrected_pvalue_df to new columns.
input_original_subset["Corrected P-Value"] = corrected_pvalue_df
input_original_subset["Hypothesis Rejected?"] = hypothesis_rej_df

#Sort -log10(p-values) in descending order i.e. smallest p-values will have largest log10 values.
#log10_corrected_pvalue_sorted = log10_corrected_pvalue.sort_values(0, ascending=False)

#Sort data frame by ascending p-values i.e. smallest to largest.
input_sorted = input_original_subset.sort_values(input_original_subset.columns[8])
#print input_sorted
#print input_sorted
#input_sorted.to_csv('SortedPvalueDataAnalysis.csv')  
print kinaseList

In [338]:
top_200=kinaseList.nlargest(200, '-Log10 Corrected P-Value')
top_200.head()

,AZ20_ctrlCV,AZ20_fold_change,AZ20_mean,AZ20_p-value,AZ20_treatCV,Kinase,Phosphosite,Substrate,control_mean,SUB_ACC_ID,p-site,-Log10 Corrected P-Value,Log2 Fold Change
309,0.025886,0.270944,9.099772e+08,4.390000e-07,0.178333,Akt2,S161,EDC3,3.358543e+09,,_S161,6.357535,-1.883933
310,0.025886,0.270944,9.099772e+08,4.390000e-07,0.178333,Akt1,S161,EDC3,3.358543e+09,,_S161,6.357535,-1.883933
995,0.061562,0.157781,1.944612e+09,7.430000e-07,0.200451,PKCD,S236,RPS6,1.232477e+10,,_S236,6.129011,-2.664007
996,0.061562,0.157781,1.944612e+09,7.430000e-07,0.200451,p90RSK,S236,RPS6,1.232477e+10,,_S236,6.129011,-2.664007
997,0.061562,0.157781,1.944612e+09,7.430000e-07,0.200451,mTOR,S236,RPS6,1.232477e+10,,_S236,6.129011,-2.664007


In [327]:
def VolcanoPlot(kinaseList):

    FC_T=1
    FC_TN=-1
    PV_T=-np.log10(0.05)

    kinaseList.loc[(kinaseList['Log2 Fold Change'] > FC_T) & (kinaseList['-Log10 Corrected P-Value'] > PV_T), 'color' ] = "Green"  # upregulated
    kinaseList.loc[(kinaseList['Log2 Fold Change'] < FC_T) & (kinaseList['-Log10 Corrected P-Value'] > PV_T), 'color' ] = "Red"   # downregulated
    kinaseList['color'].fillna('grey', inplace=True)

    output_notebook()

    category = 'Substrate'

    category_items = kinaseList[category].unique()
    title="Volcano Plot"

    #title = Inhibitor + " :Data with identified kinases"
    #feeding data into ColumnDataSource

    source = ColumnDataSource(kinaseList)

    hover = HoverTool(tooltips=[('Kinase','@Kinase'),
                                ('Substrate', '@Substrate'),
                                ('Phosphosite', '@Phosphosite'),
                                ('Fold_change', '@{Log2 Fold Change}'),
                                ('p_value', '@{-Log10 Corrected P-Value}')])

    tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool()]
    
    p = figure(tools=tools,title=title,plot_width=700,plot_height=400,toolbar_location='right',
           toolbar_sticky=False)
   
    p.scatter(x = 'Log2 Fold Change', y = '-Log10 Corrected P-Value',source=source,size=10,color='color')
   
    p_sig = Span(location=PV_T,dimension='width', line_color='black',line_dash='dashed', line_width=3)
    fold_sig_over=Span(location=FC_T,dimension='height', line_color='black',line_dash='dashed', line_width=3)
    fold_sig_under=Span(location=FC_TN,dimension='height', line_color='black',line_dash='dashed', line_width=3)

    p.add_layout(p_sig)   
    p.add_layout(fold_sig_over)   
    p.add_layout(fold_sig_under)   

    show(p)
volcano_plot=VolcanoPlot(log2FCKinase)

Loading BokehJS ...

In [329]:
def sortedDF(df):
    sorted_df=df.sort_values(by='mS', ascending=False).head(25)
    return sorted_df

sorted_df=sortedDF(calculations_df)
alt_sorted_df=sortedDF(alt_calculations_df)

In [330]:
import plotly.express as px
fig = px.bar(sorted_df, x="mS", y="Kinase", orientation='h',
             hover_data=["mS", "Kinase"],
             height=600,
             title='Kinase Substrate Enrichment (Mean Method)')
fig.show()

In [331]:
sorted_df_alt = alt_calculations_df.sort_values(by='mS', ascending=False).head(25)

fig = px.bar(sorted_df_alt, x="mS", y="Kinase", orientation='h',
             hover_data=["mS", "Kinase"],
             height=600,
             title='Kinase Substrate Enrichment (Alternative Mean Method)')
fig.show()